# DVT Explorer
Sonification of star and planet variables from the Transiting Exoplanet Survey Satellite (TESS) stored in Data Validation Time-series (DVT) files. Data: STScI.

Data download:https://archive.stsci.edu/missions/tess/tid/ 

#### ACKNOWLEDGMENT
This research includes data collected with the TESS mission, obtained from the MAST data archive at the Space Telescope Science Institute (STScI). Funding for the TESS mission is provided by the NASA Explorer Program and by the NASA Science Mission Directorate. STScI is operated by the Association of Universities for Research in Astronomy, Inc., under NASA contract NAS 5–26555.


In [1]:
from astropy.io import fits
from astropy.timeseries import TimeSeries
from astropy.timeseries import BoxLeastSquares
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import aggregate_downsample
from astropy import units as u

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import FunctionTransformer

import os
from pathlib import Path

import time
from pythonosc import udp_client


In [2]:
root = '## YOUR FOLDER PATH TO THE DOWNLOADED LIGHTCURVES ##'

In [3]:
for path, subdirs, files in os.walk(root):
    for name in files:
        print([os.path.join(path, name)])

['/Volumes/WORK/DOCTORADO_UPM/ICAD2022/SingleTIC/tess2018235142541-s0002-s0002-0000000055652896-00109_dvt.fits']


In [4]:
len(files)

1

In [5]:
file = [os.path.join(path, name)]
str = " " 
Ffile = (str.join(file))  

route = Path(Ffile)

Fname = route.with_suffix('')

In [6]:
files.sort()

In [7]:
sample = files[0:10]
sample

['tess2018235142541-s0002-s0002-0000000055652896-00109_dvt.fits']

In [8]:
curves=0
i = 0
figure = 0
for path, subdirs, files in os.walk(root):
    for name in files: 
        curves=curves+1
        file = [os.path.join(path, name)]
        str = " " 
        Ffile = (str.join(file))
        route = Path(name)
        Fname = route.with_suffix('')
        Fpng = route.with_suffix('.png')

#----------------------------------------------        
        dvt_file = Ffile
        
        with fits.open(dvt_file, mode="readonly") as hdulist:
    
            Teff = hdulist[0].header['TEFF']
            Metal = hdulist[0].header['MH']
            Radius = hdulist[0].header['RADIUS']
            Logg = hdulist[0].header['LOGG']
            Period = hdulist[1].header['TPERIOD'] 
            Freq = (1/Period)*10000

            Duration = hdulist[1].header['TDUR']
            Depth = hdulist[1].header['TDEPTH']
            
            times = hdulist[1].data['TIME']
            phases = hdulist[1].data['PHASE']
            fluxes_init = hdulist[1].data['LC_INIT']
            model_fluxes_init = hdulist[1].data['MODEL_INIT']
            
        
        if Duration >= 8:
            figure = 5
        if (Duration >= 4) and (Duration < 8):
            figure = 4
        if (Duration > 2) and (Duration <= 4):
            figure = 3
        if Duration <= (2):
            figure = 2
        
        
        sort_indexes = np.argsort(phases)

        fig, ax = plt.subplots(figsize=(12,4))

        ax.plot(phases[sort_indexes], fluxes_init[sort_indexes], 'go', markersize=1)

        ax.plot(phases[sort_indexes], model_fluxes_init[sort_indexes], '-w')

        fig.suptitle("{object}. Folded lightcurve and transit model".format(object= hdulist[0].header['OBJECT']), color = 'w')
        
        ax.set_ylabel("Flux (relative)")
        ax.set_xlabel("Orbital Phase")
        ax.xaxis.label.set_color('w')
        ax.yaxis.label.set_color('w')
        ax.tick_params(axis='x', colors='w')
        ax.tick_params(axis='y', colors='w')
        ax.spines['bottom'].set_color('w')
        ax.spines['top'].set_color('w') 
        ax.spines['right'].set_color('w')
        ax.spines['left'].set_color('w')
     
        plt.savefig('DVT.png', transparent=True)
        
        plt.close(fig)
      
        #---------------Sending OSC
        print(hdulist[0].header['OBJECT'])
        client_s = udp_client.SimpleUDPClient("127.0.0.1", 9989)
        client_s.send_message("/s", 1)

        client0 = udp_client.SimpleUDPClient("127.0.0.1", 9990)   
        client0.send_message("/Teff", Teff)
        print("Teff =", Teff)

        client1 = udp_client.SimpleUDPClient("127.0.0.1", 9991) 
        client1.send_message("/Metal", Metal)
        print("Metallicity =", Metal)

        client2 = udp_client.SimpleUDPClient("127.0.0.1", 9992) 
        client2.send_message("/Radius", Radius)
        print("Radius =", Radius)

        client3 = udp_client.SimpleUDPClient("127.0.0.1", 9993) #Surface gravity
        client3.send_message("/Logg", Logg)
        print("Log(g) =", Logg)
    
        client4 = udp_client.SimpleUDPClient("127.0.0.1", 9994) #Duration
        client4.send_message("/Dur", Duration)
        print("Duration =", Duration)
        
        client5 = udp_client.SimpleUDPClient("127.0.0.1", 9995) #Amplitude
        client5.send_message("/Amp", Depth)
        print("Depth =", Depth)
        
        client6 = udp_client.SimpleUDPClient("127.0.0.1", 9996) 
        client6.send_message("/Freq", Freq)
        print("Period =", Period)
        print("Frequency =", Freq)
        
        print("--------------------------")



        i += 1
        time.sleep(figure)
            
print ("Files represented:",curves, "curves");



TIC 55652896
Teff = 5025.77001953
Metallicity = 0.31632501
Radius = 0.71520799
Log(g) = 4.6634798
Duration = 5.399625875925155
Depth = 3438.7793282869966
Period = 10.86195221157628
Frequency = 920.6448164393835
--------------------------
Files represented: 1 curves


## Adding VO tools and querys

In [9]:
import ipyaladin.aladin_widget as ipyal

aladin = ipyal.Aladin(target= hdulist[0].header['OBJECT'], fov=0.15, survey='P/DSS2/color')
aladin

Aladin(fov=0.15, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'o…

https://astroquery.readthedocs.io/en/latest/index.html

In [10]:
from astroquery.simbad import Simbad
result_table = Simbad.query_object(hdulist[0].header['OBJECT'])
result_table.pprint(show_unit=True)

MAIN_ID       RA           DEC      ...     COO_BIBCODE     SCRIPT_NUMBER_ID
           "h:m:s"       "d:m:s"    ...                                     
------- ------------- ------------- ... ------------------- ----------------
TOI-216 04 55 55.2548 -63 15 36.313 ... 2020yCat.1350....0G                1


In [11]:
from astroquery.vizier import Vizier
result = Vizier.query_object(hdulist[0].header['OBJECT'])
print(result)

TableList with 83 tables:
	'0:I/221/lmc' with 8 column(s) and 3 row(s) 
	'1:I/252/out' with 8 column(s) and 39 row(s) 
	'2:I/271/out' with 11 column(s) and 21 row(s) 
	'3:I/284/out' with 14 column(s) and 50 row(s) 
	'4:I/289/out' with 13 column(s) and 3 row(s) 
	'5:I/297/out' with 19 column(s) and 50 row(s) 
	'6:I/305/out' with 11 column(s) and 28 row(s) 
	'7:I/317/sample' with 13 column(s) and 50 row(s) 
	'8:I/319/xpm' with 25 column(s) and 13 row(s) 
	'9:I/320/spm4' with 21 column(s) and 9 row(s) 
	'10:I/322A/out' with 24 column(s) and 5 row(s) 
	'11:I/324/igsl3' with 18 column(s) and 50 row(s) 
	'12:I/337/gaia' with 14 column(s) and 50 row(s) 
	'13:I/339/hsoy' with 18 column(s) and 43 row(s) 
	'14:I/340/ucac5' with 20 column(s) and 5 row(s) 
	'15:I/345/gaia2' with 32 column(s) and 50 row(s) 
	'16:I/347/gaia2dis' with 9 column(s) and 50 row(s) 
	'17:I/348/catalog' with 35 column(s) and 1 row(s) 
	'18:I/349/starhorse' with 19 column(s) and 11 row(s) 
	'19:I/350/gaiaedr3' with 39 colum

In [12]:
interesting_table = result['B/eso/eso_arc']
print(interesting_table)

TelID       _tab1_6         tExp   Ask     Target     RAJ2000     DEJ2000  
               s             s                        "h:m:s"     "d:m:s"  
----- ------------------- ------- ----- ----------- ----------- -----------
   20 2018-11-28T02:47:53   900.0 Image     TOI-216 04 55 48.17 -63 14 47.4
   20 2018-11-28T05:52:46   900.0 Image     TOI-216 04 55 47.11 -63 14 46.2
   30 2018-12-13T05:23:44  1800.0 Image     TOI-216 04 55 54.92 -63 15 45.8
   30 2018-12-13T05:54:44  1800.0 Image     TOI-216 04 55 54.88 -63 15 45.0
   20 2018-11-27T04:50:09   900.0 Image     TOI-216 04 55 47.13 -63 14 48.0
   30 2018-12-12T06:05:29  1800.0 Image     TOI-216 04 55 54.73 -63 15 45.6
  ...                 ...     ...   ...         ...         ...         ...
   91 2019-01-25T01:57:45     9.0 Image TIC55652896 04 55 54.83 -63 15 42.0
   91 2019-01-25T01:58:10     9.0 Image TIC55652896 04 55 55.13 -63 15 42.0
   91 2019-01-25T01:58:34     9.0 Image TIC55652896 04 55 55.42 -63 15 42.0
   91 2019-0

In [13]:
from astroquery.mast import Observations

obs_table = Observations.query_object(hdulist[0].header['OBJECT'],radius=".02 deg")
data_products_by_obs = Observations.get_product_list(obs_table[0:2])
print(data_products_by_obs)  

 obsID   obs_collection dataproduct_type ... dataRights calib_level
-------- -------------- ---------------- ... ---------- -----------
60853722           TESS            image ...     PUBLIC           2
60853722           TESS            image ...     PUBLIC           1
60847653           TESS            image ...     PUBLIC           2
60847653           TESS            image ...     PUBLIC           1
60848599           TESS            image ...     PUBLIC           2
60848599           TESS            image ...     PUBLIC           1
60846776           TESS            image ...     PUBLIC           2
     ...            ...              ... ...        ...         ...
60899567           TESS            image ...     PUBLIC           1
60893483           TESS            image ...     PUBLIC           2
60893483           TESS            image ...     PUBLIC           1
60885107           TESS            image ...     PUBLIC           2
60885107           TESS            image ...    

Stop

In [14]:
client_s.send_message("/s", 0)
client0.send_message("/Teff", 0)
client1.send_message("/Metal", 0)
client2.send_message("/Radius", 0)
client3.send_message("/Logg",0)
client4.send_message("/Dur", 0)
client5.send_message("/Amp", 0)
client6.send_message("/Freq", 0)


In [1]:
import session_info
session_info.show()